In [6]:
import pandas as pd
import yfinance as yf
import ipywidgets as widgets
from ipywidgets import interact
import statsmodels.api as sm
import seaborn as sns
import warnings

from src.datafeed import get_data

sns.set_style("whitegrid")
warnings.filterwarnings('ignore')

In [7]:
# get prices until today
today = pd.Timestamp("now").strftime("%Y-%m-%d")
price = get_data(today)

print(price.head())  # Display the first few rows of the DataFrame


                AAPL        CVX        DAL  GM          GS        JPM  \
Date                                                                    
2010-01-04  6.478997  45.479145   9.917485 NaN  137.658157  29.852793   
2010-01-05  6.490200  45.801273  10.695327 NaN  140.091843  30.431046   
2010-01-06  6.386964  45.807030  10.704165 NaN  138.596649  30.598246   
2010-01-07  6.375158  45.634449  11.216832 NaN  141.308762  31.204363   
2010-01-08  6.417541  45.714996  11.022376 NaN  138.636368  31.127739   

                  LUV       MSFT        SPY  TSLA        XOM        SP500  
Date                                                                       
2010-01-04  10.143021  23.474915  87.483231   NaN  40.611740  1132.989990  
2010-01-05  10.678296  23.482513  87.714783   NaN  40.770309  1136.520020  
2010-01-06  10.360757  23.338396  87.776543   NaN  41.122684  1137.140015  
2010-01-07  10.514990  23.095684  88.147095   NaN  40.993492  1141.689941  
2010-01-08  10.279104  23.254965

In [8]:
# average returns
mu = price.pct_change().dropna().mean().rename("mu") * 252 * 100

In [9]:
# main app
# Dropdown widget for frequency selection
frequency_selector = widgets.Dropdown(
    options=[('1 day', '1D'), ('1 month', '1M'), ('1 year', '1Y')],
    value='1D',
    description='Frequency:',
)

In [11]:
display(f"Sample from 2010-01-01 to {today}")

@interact
def app(frequency=frequency_selector):
    """"""
    r = price.resample(frequency).last().pct_change().dropna()
    x = sm.add_constant(r.pop("SP500"))

    # ols betas
    beta = {}
    for _c_name, _c in r.items():
        beta[_c_name] = sm.OLS(endog=_c, exog=x, missing="drop")\
            .fit().params[1]
    
    beta = pd.Series(beta, name="beta")
    to_plot = pd.concat((beta, mu), axis=1)

    sns.regplot(data=to_plot, x="beta", y="mu", ci=None)
        

'Sample from 2010-01-01 to 2023-11-27'

interactive(children=(Dropdown(description='Frequency:', index=2, options=(('1 day', '1D'), ('1 month', '1M'),…